## Pipeline: Silver to Gold

This pipeline consumes trusted Silver data and applies business logic, aggregations, and metric calculations to create optimized Gold datasets designed for analytics, dashboards, and executive reporting.

In [57]:
df_silver = spark.read.format("delta").load("abfss://silver@lablicitacoessa.dfs.core.windows.net/Licitacoes_Gov/")

display(df_silver)

StatementMeta(sparkpool1, 1, 58, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 30ba4adc-165c-41da-ae5e-d6b35bc15695)